In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.insert(0, "../")

In [2]:
from tqdm.auto import tqdm
import torch
import json
from pathlib import Path
from nohomers.ml.lightweight_gan_trainer import get_trainer, gen_images_and_manifest, gen_interpolation_videos, GeneratedRef
from nohomers.ml.cleaner import load_cleaner
from uuid import uuid4
from PIL import Image
import torchvision
import tempfile
from torchvision import transforms
import numpy as np
import ffmpeg
import shutil
import copy
import random

In [3]:
trainer = get_trainer(
    models_dir="/mnt/evo/projects/metapedia/tmp/stylegan2/models", 
    name="simpsons_large_cleaned_nobackground_1024_augall03_sle_res64",
    load_from=40,
)

loading from version 0.12.4


In [4]:
cleaner = load_cleaner(
    Path("/mnt/evo/projects/nohomers/cleaners/simpsons_large_cleaned_nobackground_1024_augall03_sle_res64-40/cleaner.pkl"),
    latent_dim=256,
).cuda()

In [4]:
output_dir = Path("/mnt/evo/projects/nohomers/assets/simpsons_large_cleaned_nobackground_1024_augall03_sle_res64-40-p88")

In [6]:
shutil.rmtree(output_dir, ignore_errors=True, onerror=None)
output_dir.mkdir(parents=True, exist_ok=True)

manifest = gen_images_and_manifest(
    trainer, 
    output_dir,
    num=20_000,
    batch_size=600,
    cleaner=cleaner,
    clean_threshold=0.63,
)

In [5]:
manifest_path = Path(output_dir) / "image_manifest.json"

In [8]:
with open(manifest_path, "w") as f:
    json.dump([e.to_dict() for e in manifest], f)

In [6]:
with open(manifest_path, "r") as f:
    manifest = [GeneratedRef.from_dict(e) for e in json.load(f)]

In [7]:
len(manifest)

20000

In [9]:
video_manifest = gen_interpolation_videos(
    trainer, manifest, output_dir, per_edge=3, video_duration=2.0, batch_size=225, video_fps=30,
    manifest_save_path=Path(output_dir / "manifest.json"), save_every=1000,
)

In [ ]:
manifest_path = Path(output_dir) / "manifest.json"
with open(manifest_path, "w") as f:
    json.dump([e.to_dict() for e in video_manifest], f)